In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from collections import defaultdict
import spacy
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
import optuna
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors


c:\Users\didia\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [36]:
#carregar dataset
df = pd.read_csv('factnews_dataset.csv')
df.isnull().sum()

file          0
id_sente      0
id_article    0
domain        0
year          0
sentences     0
classe        0
dtype: int64

# Pré-processamento

divisão dos dados

In [ ]:
'''
#criar grupo treino, validação e teste
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['classe'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['classe'])

#guardar conjuntos em memória
train_df.to_csv("train.csv", index=False) 
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)
'''

balanceamento dos dados

In [55]:
train= pd.read_csv('train.csv')
val= pd.read_csv('val.csv')
test= pd.read_csv('test.csv')

y_test= test['classe']
y_train= train['classe']
y_val=val['classe']
classes = test['classe'].unique()

In [56]:
# Contagem das classes em cada conjunto
print ("conjunto de treino: ",y_train.value_counts().sort_index())
print("\nconjunto de validação: ",y_val.value_counts().sort_index())
print("\nconjunto de teste: ", y_test.value_counts().sort_index())



conjunto de treino:  classe
-1     779
 0    2375
 1     312
Name: count, dtype: int64

conjunto de validação:  classe
-1    195
 0    594
 1     78
Name: count, dtype: int64

conjunto de teste:  classe
-1     417
 0    1273
 1     168
Name: count, dtype: int64


In [57]:
from sklearn.utils import resample

# Separar cada classe
class_min = train[train['classe'] == -1]
class_med = train[train['classe'] == 1]
class_max = train[train['classe'] == 0]

# Número da classe maior
n_samples = len(class_max)

# Replicar as classes menores até o tamanho da maior
class_min_upsampled = resample(class_min, 
                            replace=True, 
                            n_samples=n_samples, 
                            random_state=42)

class_med_upsampled = resample(class_med, 
                            replace=True, 
                            n_samples=n_samples, 
                            random_state=42)

train = pd.concat([class_min_upsampled, class_med_upsampled, class_max])

# baralhar o conjunto de treino
train = train.sample(frac=1, random_state=42).reset_index(drop=True)

print(train['classe'].value_counts())


classe
 1    2375
 0    2375
-1    2375
Name: count, dtype: int64


tokeinização

In [84]:

stop_words = set(stopwords.words('portuguese'))

#criar coluna tokens
train['tokens'] = train['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
val['tokens']   = val['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
test['tokens'] = test['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))

#remover stopwords
train['tokens'] = train['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
val['tokens']   = val['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
test['tokens']  = test['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])

#matizar
#nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
val['stems']   = val['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
test['stems'] = test['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
print(train['stems'].head())

y_test= test['classe']
y_train= train['classe']
y_val=val['classe']
classes = test['classe'].unique()


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


0    [qued, ocorr, aliment, pux, cord, sent, contr,...
1    [repercuss, nega, cas, fez, bolsonar, ,, candi...
2    [brasil, lav, alm, após, decepcion, empat, col...
3    [final, encontr, ,, sen, tucan, diss, esp, gov...
4    [fiz, compromiss, líd, bas, hoj, líd, opos, ,,...
Name: stems, dtype: object


# Construção dos modelos de representação

## TF-IDF - fred

In [ ]:
def TF_IDF():
    tf_vect = TfidfVectorizer ( ngram_range =(1 ,2) , min_df =3 ,
    max_df =0.5 , max_features =500)
    p_tf = tf_vect . fit_transform ( docs )

## CountVectorizer -didi

In [95]:
def count_vec(train,val,test):
    c_vect = CountVectorizer(token_pattern=r"(?u)\b\w+\b|[^\w\s]", ngram_range=(1,1), strip_accents='unicode',max_features=1000)

    #juntar os tokens matizados em frases novvamente 
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))
    
    #print("Text train:\n", text_train.head())

    # Transformar em vetores
    X_train = c_vect.fit_transform(text_train)
    X_val   = c_vect.transform(text_val)
    X_test  = c_vect.transform(text_test)

    return X_train, X_val, X_test

In [96]:
X_train_count_vec, X_val_count_vec, X_test_count_vec = count_vec(train,val,test)

## Glove -fred

## WORD2VEC -anaana

fontes : 
-   https://sites.google.com/view/nilc-usp/resources-and-tools?authuser=0
-   https://huggingface.co/nilc-nlp/word2vec-skip-gram-300d

In [ ]:
#%pip install huggingface_hub
#%pip install hf_hub_download
#%pip install safetensors
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file

path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                       filename="embeddings.safetensors")

data = load_file(path)
vectors = data["embeddings"]

vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                             filename="vocab.txt")
with open(vocab_path) as f:
    vocab = [w.strip() for w in f]

print(vectors.shape)

model = KeyedVectors(vector_size=vectors.shape[1])
model.add_vectors(vocab, vectors)
model.fill_norms()

def vetor_frase(model, frase):
    vectors = [model[w] for w in frase.lower().split() if w in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)
    
def conjunto_para_vetores(model, frases):
    """
    Recebe uma lista de frases e devolve uma matriz NumPy
    com as médias dos vetores.
    """
    return np.vstack([vetor_frase(model, f) for f in frases])

    
# Transformar todos os conjuntos
X_train = np.vstack([vetor_frase(model, f) for f in frases])(model, train['stems'])
X_val   = np.vstack([vetor_frase(model, f) for f in frases])(model, val['stems'])
X_test  = np.vstack([vetor_frase(model, f) for f in frases])(model, test['stems'])

print("X_train.shape:", X_train.shape)
print("X_val.shape:", X_val.shape)
print("X_test.shape:", X_test.shape)

embeddings.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

c:\Users\didia\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:121: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\didia\.cache\huggingface\hub\models--nilc-nlp--word2vec-skip-gram-300d. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt: 0.00B [00:00, ?B/s]

(929606, 300)


In [ ]:
'''from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
certos = (test_df['classe'] == test_df['Previsto4']).sum()
total = len(test_df)

matriz = confusion_matrix(test_df['classe'], test_df['Previsto4'], labels=classes)

disp = ConfusionMatrixDisplay(confusion_matrix=matriz, display_labels=classes)
disp.plot()
plt.title("Matriz de confusão do teste")
plt.show()

from sklearn.metrics import classification_report

report = classification_report(test_df['classe'], test_df['Previsto4'], labels=classes, digits=4)
print(report)'''

# Aplicar modelos de classificação

## Naive Bayes

In [97]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, confusion_matrix, classification_report

def Naive_Bayes(X_train, y_train, X_val, y_val, X_test, alphas):
    best_alpha = None
    best_f1 = 0

    # Procurar o melhor alpha com o conjunto de validação
    for alpha in alphas:
        clf = MultinomialNB(alpha=alpha)
        clf.fit(X_train, y_train)
        y_val_pred = clf.predict(X_val)
        f1 = f1_score(y_val, y_val_pred, average='weighted')
        if f1 > best_f1:
            best_f1 = f1
            best_alpha = alpha
    
    print(f"Melhor alpha encontrado: {best_alpha} (F1 val = {best_f1:.4f})")

    # Treinar o modelo final 
    final_clf = MultinomialNB(alpha=best_alpha)
    final_clf.fit(X_train, y_train)

    y_test_pred = final_clf.predict(X_test)

    return y_test_pred 


Count Vectorizer

In [99]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NB= Naive_Bayes(X_train_count_vec, y_train, X_val_count_vec,y_val , X_test_count_vec, 
                                    alphas=[0.01, 0.1, 0.5, 1.0, 3.0,5.0, 6.0, 7.0]) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NB))

#TF-IDF

#Word2vec

#Glove


----COUNT VECTORIZER ----
Melhor alpha encontrado: 0.5 (F1 val = 0.7702)
              precision    recall  f1-score   support

           1       0.65      0.77      0.71       417
           0       0.87      0.72      0.79      1273
          -1       0.23      0.43      0.30       168

    accuracy                           0.71      1858
   macro avg       0.58      0.64      0.60      1858
weighted avg       0.76      0.71      0.73      1858

[[323  65  29]
 [149 918 206]
 [ 26  70  72]]


## SVM

In [90]:
def objective(trial, X_train, X_val, y_train, y_val):

    #Definir hiperparâmetros a testar 
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    #Criar modelo SVM 
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)

    #treinar modelo com conjunto de treino
    model.fit(X_train, y_train)

    #prever no conjunto de validação
    y_pred = model.predict(X_val)

    #calcular f1
    f1 = f1_score(y_val, y_pred, average='macro')

    return f1

def svm(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    # Chamar o Optuna para otimizar os parâmetros
    objeto_para_otimizar = optuna.create_study(direction='maximize')
    objeto_para_otimizar.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False )

    print("Melhores hiperparâmetros encontrados:")
    print(objeto_para_otimizar.best_params)

    # Treinar o melhor modelo
    best_model = SVC(**objeto_para_otimizar.best_params)
    best_model.fit(X_train, y_train)

    #  Avaliar no conjunto de teste
    y_pred_test = best_model.predict(X_test)

    return y_pred_test

In [81]:
print ("----COUNT VECTORIZER ----")
count_vec_predicted_SVM= svm(X_train_count_vec, X_val_count_vec, X_test_count_vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_SVM))

[I 2025-11-09 11:38:12,643] A new study created in memory with name: no-name-62a4a033-e679-4d40-a2d8-b9747932cc6e


----COUNT VECTORIZER ----


[I 2025-11-09 11:38:19,820] Trial 6 finished with value: 0.5552783473836106 and parameters: {'C': 0.8581046207222348, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 6 with value: 0.5552783473836106.
[I 2025-11-09 11:38:26,423] Trial 7 finished with value: 0.5492352355448445 and parameters: {'C': 4.916290647647583, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 6 with value: 0.5552783473836106.
[I 2025-11-09 11:38:27,187] Trial 0 finished with value: 0.5878419723180776 and parameters: {'C': 0.026920245231017655, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 0 with value: 0.5878419723180776.
[I 2025-11-09 11:38:29,974] Trial 3 finished with value: 0.12241054613935969 and parameters: {'C': 7.848718233911146, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 0 with value: 0.5878419723180776.
[I 2025-11-09 11:38:31,637] Trial 2 finished with value: 0.3044757450677113 and parameters: {'C': 0.0534320265240413, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 wi

Melhores hiperparâmetros encontrados:
{'C': 0.01161857843662566, 'kernel': 'linear', 'gamma': 'scale'}
              precision    recall  f1-score   support

           1       0.52      0.65      0.58       417
           0       0.84      0.70      0.77      1273
          -1       0.27      0.42      0.33       168

    accuracy                           0.67      1858
   macro avg       0.54      0.59      0.56      1858
weighted avg       0.71      0.67      0.68      1858

[[272 109  36]
 [220 897 156]
 [ 33  65  70]]


## KNN

In [91]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def KNN(X_train, y_train, X_val, y_val, X_test):
    k_values = range(1, 200,10)  # Determinar  intervalo de valores a testar para k
    best_k = 1
    best_accuracy = 0
    
    k_list = []
    accuracy_list = [] 

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)  
        knn.fit(X_train, y_train)

        y_pred_KNN = knn.predict(X_val) 
        accuracy = accuracy_score(y_val, y_pred_KNN)  

        k_list.append(k)
        accuracy_list.append(accuracy)  

        if accuracy > best_accuracy: 
            best_accuracy = accuracy
            best_k = k

    knn = KNeighborsClassifier(n_neighbors=best_k)  
    knn.fit(X_train, y_train)

    y_pred_KNN= knn.predict(X_test) 
    print(f"Best k: {best_k}")

    return y_pred_KNN


In [100]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= KNN(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

#TF-IDF

#Word2vec

#Glove


----COUNT VECTORIZER ----
Best k: 41
              precision    recall  f1-score   support

           1       0.43      0.84      0.57       417
           0       0.83      0.63      0.71      1273
          -1       0.12      0.06      0.08       168

    accuracy                           0.62      1858
   macro avg       0.46      0.51      0.45      1858
weighted avg       0.68      0.62      0.62      1858

[[352  55  10]
 [414 796  63]
 [ 53 105  10]]


## Decison Tree

In [93]:
#Criar um classificador de árvore de decisão baseado em entropia
from sklearn.tree import DecisionTreeClassifier

def decision_tree(X_train, y_train, X_val, y_val, X_test):
    alphas = np.linspace(0.000, 0.01, 20)  # Testar 20 valores entre 0 e 0.01
    best_alpha = 0
    best_acc = 0

    for alpha in alphas:
        clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=alpha, random_state=42)
        clf.fit(X_train, y_train)
        y_pred_val = clf.predict(X_val)
        acc = accuracy_score(y_val, y_pred_val)

        if acc > best_acc:
            best_acc = acc
            best_alpha = alpha

    print(f"Melhor alpha: {best_alpha:.4f} ")

    # Treinar modelo final com o melhor alpha
    final_clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=best_alpha, random_state=42)
    final_clf.fit(X_train, y_train)
    y_pred_test = final_clf.predict(X_test)

    return y_pred_test

In [94]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_DT= decision_tree(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_DT))

#TF-IDF

#Word2vec

#Glove


----COUNT VECTORIZER ----
Melhor alpha: 0.0005 
              precision    recall  f1-score   support

           1       0.66      0.64      0.65       417
           0       0.81      0.82      0.81      1273
          -1       0.19      0.17      0.18       168

    accuracy                           0.72      1858
   macro avg       0.55      0.54      0.55      1858
weighted avg       0.72      0.72      0.72      1858

[[ 268  132   17]
 [ 120 1047  106]
 [  19  121   28]]
